In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler

import numpy as np, pandas as pd
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import time
from datetime import datetime

from torch.utils.data import DataLoader, TensorDataset

import math
from tqdm import tqdm
import copy

In [2]:
df_tr = pd.read_csv("train.csv")


# Over every single 
def polyline_to_trip_duration(polyline):
  return max(polyline.count("[") - 2, 0) * 15

df_tr["LEN"] = df_tr["POLYLINE"].apply(polyline_to_trip_duration)

from datetime import datetime
def parse_time(x):
  # We are using python's builtin datetime library
  # https://docs.python.org/3/library/datetime.html#datetime.date.fromtimestamp

  # Each x is essentially a 1 row, 1 column pandas Series
  dt = datetime.fromtimestamp(x["TIMESTAMP"])
  return dt.year, dt.month, dt.day, dt.hour, dt.weekday()

# Because we are assigning multiple values at a time, we need to "expand" our computed (year, month, day, hour, weekday) tuples on 
# the column axis, or axis 1
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.apply.html
df_tr[["YR", "MON", "DAY", "HR", "WK"]] = df_tr[["TIMESTAMP"]].apply(parse_time, axis=1, result_type="expand")

def mean_absolute_error(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)  # Convert y_true to float32
    return tf.reduce_mean(tf.abs(y_pred - y_true))
      
def root_mean_squared_error(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)  # Convert y_true to float32
    return tf.sqrt(tf.reduce_mean(tf.square(y_pred - y_true)))

def get_days_in_month(year, month):
    if month == 12:
        return (datetime(year+1, 1, 1) - datetime(year, month, 1)).days
    else:
        return (datetime(year, month+1, 1) - datetime(year, month, 1)).days
    

gps_location_df = pd.read_csv('metaData_taxistandsID_name_GPSlocation.csv')

# Create a dictionary for each ORIGIN_STAND, where key is the stand id and value is a dic (Latitude, Longitude)
location_dict = gps_location_df.set_index('ID')[['Latitude', 'Longitude']].T.apply(tuple).to_dict()

df_tr['Latitude'] = df_tr['ORIGIN_STAND'].map(lambda x: location_dict.get(x, {}).get('Latitude', np.nan))
df_tr['Longitude'] = df_tr['ORIGIN_STAND'].map(lambda x: location_dict.get(x, {}).get('Longitude', np.nan))

In [3]:
df_test = pd.read_csv("test_public.csv")

taxi_test_ids = []

for id in df_test["TAXI_ID"]:
    if id not in taxi_test_ids:
        taxi_test_ids.append(id)

In [4]:
taxi_test_ids_dic = {}
for ele in taxi_test_ids:
    taxi_test_ids_dic[ele] = ele

In [5]:
df_test = pd.read_csv("test_public.csv")
l2 = []
for ele in df_test['ORIGIN_CALL']:
    if math.isnan(ele):
        continue
    l2.append(ele)

In [6]:
origincall_dic = {}

for ele in l2:
    if df_tr['ORIGIN_CALL'].value_counts().get(ele, 0) >= 1000:
        origincall_dic[ele] = ele

In [12]:
df = df_tr.copy()

df['ORIGIN_CALL'].fillna(0, inplace=True)
df['ORIGIN_STAND'].fillna(0, inplace=True)
df['Latitude'].fillna(0, inplace=True)
df['Longitude'].fillna(0, inplace=True)
df = df[df['MISSING_DATA'] != True]
df['TAXI_ID'] = df['TAXI_ID'].map(lambda x: taxi_test_ids_dic.get(x, 0))
# df['ORIGIN_CALL'] = df['ORIGIN_CALL'].map(lambda x: origincall_dic.get(x, 0))

string_features = ['CALL_TYPE', 'ORIGIN_STAND', 'TAXI_ID', 'DAY_TYPE', 'YR', 'MON', 'DAY', 'HR', 'WK'] 

string_transformers = [('encode_' + feature, OneHotEncoder(), [feature]) for feature in string_features]

numerical_transformers = [('scale_' + feature, StandardScaler(), [feature]) for feature in ['Latitude', 'Longitude']]

transformers = string_transformers + numerical_transformers

preprocessor = ColumnTransformer(transformers=transformers)

X_train = df.drop(columns=['ORIGIN_CALL', 'TRIP_ID', 'TIMESTAMP', 'MISSING_DATA','POLYLINE','LEN'])
y = df['LEN']


X_train_preprocessed = preprocessor.fit_transform(X_train).toarray()  # convert to dense array here

train_data = TensorDataset(torch.tensor(X_train_preprocessed, dtype=torch.float), torch.tensor(y.values, dtype=torch.float))

batch_size = 32

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

class Net(nn.Module):
    def __init__(self, input_dim):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

input_dim = X_train_preprocessed.shape[1]
model = Net(input_dim)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

for epoch in range(5):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        mse_loss = criterion(outputs, labels.view(-1, 1))
        rmse_loss = torch.sqrt(mse_loss)  # take the square root of the MSE to get the RMSE
        rmse_loss.backward()  # use the RMSE for backpropagation
        optimizer.step()

        # print statistics
        running_loss += mse_loss.item()  # this is still the MSE, not the RMSE
    avg_mse = running_loss / (i+1)
    rmse = torch.sqrt(torch.tensor(avg_mse))
    print(f'Epoch: {epoch+1}, RMSE Loss: {rmse}')

print('Finished Training')


Epoch: 1, RMSE Loss: 669.4746704101562
Epoch: 2, RMSE Loss: 665.7886962890625
Epoch: 3, RMSE Loss: 664.1683959960938
Epoch: 4, RMSE Loss: 663.0091552734375
Epoch: 5, RMSE Loss: 662.0946044921875
Finished Training


In [14]:
# # switch the model to evaluation mode
# model.eval()

# with torch.no_grad():  # do not compute gradient within this context
#     test_loss = 0.0
#     for i, data in enumerate(test_loader, 0):
#         inputs, labels = data
#         outputs = model(inputs)
#         loss = criterion(outputs, labels.view(-1, 1))
#         test_loss += loss.item()

#     # RMSE is just the square root of MSE
#     test_rmse = math.sqrt(test_loss / len(test_loader))
#     print(f'Test RMSE: {test_rmse}')


ValueError: too many values to unpack (expected 2)

In [ ]:
# batch size 32 lr 0.01 Test RMSE: 719.92045162663 Test RMSE: 684.8386458251686 Test RMSE: 643.101539182635
# batch size 64 lr 0.01 Test RMSE: 701.6253132258208
# batch size 64 lr 0.001 Test RMSE: 640.0380211310754

In [13]:
df_test = pd.read_csv("test_public.csv")
df_test[["YR", "MON", "DAY", "HR", "WK"]] = df_test[["TIMESTAMP"]].apply(parse_time, axis=1, result_type="expand")
df_test['Latitude'] = df_test['ORIGIN_STAND'].map(lambda x: location_dict.get(x, {}).get('Latitude', np.nan))
df_test['Longitude'] = df_test['ORIGIN_STAND'].map(lambda x: location_dict.get(x, {}).get('Longitude', np.nan))
df_test['ORIGIN_CALL'].fillna(0, inplace=True)
df_test['ORIGIN_STAND'].fillna(0, inplace=True)
df_test['Latitude'].fillna(0, inplace=True)
df_test['Longitude'].fillna(0, inplace=True)
df_test = df_test[df_test['MISSING_DATA'] != True]
# df_test['ORIGIN_CALL'] = df_test['ORIGIN_CALL'].map(lambda x: origincall_dic.get(x, 0))


X = df_test.drop(columns=['ORIGIN_CALL', 'TRIP_ID', 'TIMESTAMP', 'MISSING_DATA'])

X_train_preprocessed = preprocessor.fit_transform(X_train).toarray()  # convert to dense array here
X_test_preprocessed = preprocessor.transform(X).toarray()  # convert to dense array here

test_data = torch.tensor(X_test_preprocessed, dtype=torch.float)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

model.eval()
predictions = []

with torch.no_grad():
    for inputs in test_loader:
        outputs = model(inputs)
        predictions.extend(outputs.numpy())

predictions = [arr.item() for arr in predictions]

# Sample submission file that is given on kaggle
df_test = pd.read_csv("test_public.csv")
df_sample = pd.read_csv("sampleSubmission.csv")

df_sample["TRAVEL_TIME"] = predictions

df_sample.to_csv("my_pred.csv", index=None)

In [27]:
df_test = pd.read_csv("test_public.csv")
df_test[["YR", "MON", "DAY", "HR", "WK"]] = df_test[["TIMESTAMP"]].apply(parse_time, axis=1, result_type="expand")
df_test['Latitude'] = df_test['ORIGIN_STAND'].map(lambda x: location_dict.get(x, {}).get('Latitude', np.nan))
df_test['Longitude'] = df_test['ORIGIN_STAND'].map(lambda x: location_dict.get(x, {}).get('Longitude', np.nan))
df_test['ORIGIN_CALL'].fillna(0, inplace=True)
df_test['ORIGIN_STAND'].fillna(0, inplace=True)
df_test['Latitude'].fillna(0, inplace=True)
df_test['Longitude'].fillna(0, inplace=True)
df_test = df_test[df_test['MISSING_DATA'] != True]

df = df_tr.copy()
df['ORIGIN_CALL'].fillna(0, inplace=True)
df['ORIGIN_STAND'].fillna(0, inplace=True)
df['Latitude'].fillna(0, inplace=True)
df['Longitude'].fillna(0, inplace=True)
df = df[df['MISSING_DATA'] != True]
taxi_list = list()
for ele in df_test['TAXI_ID']:
    taxi_list.append(ele)
ids = pd.Index(taxi_list, dtype='int64')
dfs = {taxi_id: df[df["TAXI_ID"] == taxi_id] for taxi_id in ids}
dfs['others'] = df[~df["TAXI_ID"].isin(ids)]

dfs_test = {taxi_id: df_test[df_test["TAXI_ID"] == taxi_id] for taxi_id in ids}

In [31]:
df_sample = pd.read_csv("sampleSubmission.csv")

total = {}

for i in tqdm(range(len(taxi_list))):
    taxi_id = taxi_list[i]
    print(f'taxi_id: {taxi_id},index: {i}')
    df = dfs[taxi_id]
    
    string_features = ['CALL_TYPE', 'DAY_TYPE', 'YR', 'MON', 'DAY', 'HR', 'WK'] 

    string_transformers = [('encode_' + feature, OneHotEncoder(handle_unknown='ignore'), [feature]) for feature in string_features]

    numerical_transformers = [('scale_' + feature, StandardScaler(), [feature]) for feature in ['Latitude', 'Longitude']]

    transformers = string_transformers + numerical_transformers

    preprocessor = ColumnTransformer(transformers=transformers)

    X = df.drop(columns=['TRIP_ID','ORIGIN_CALL','ORIGIN_STAND', 'TAXI_ID', 'TIMESTAMP', 'MISSING_DATA','POLYLINE','LEN'])
    y = df['LEN']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.001)

    X_train_preprocessed = preprocessor.fit_transform(X_train).toarray()  # convert to dense array here
    X_test_preprocessed = preprocessor.transform(X_test).toarray()  # convert to dense array here

    # Create dataloaders
    train_data = TensorDataset(torch.tensor(X_train_preprocessed, dtype=torch.float), torch.tensor(y_train.values, dtype=torch.float))
    test_data = TensorDataset(torch.tensor(X_test_preprocessed, dtype=torch.float), torch.tensor(y_test.values, dtype=torch.float))

    batch_size = 32

    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

    class Net(nn.Module):
        def __init__(self, input_dim):
            super(Net, self).__init__()
            self.fc1 = nn.Linear(input_dim, 64)
            self.fc2 = nn.Linear(64, 32)
            self.fc3 = nn.Linear(32, 1)

        def forward(self, x):
            x = torch.relu(self.fc1(x))
            x = torch.relu(self.fc2(x))
            return self.fc3(x)

    input_dim = X_train_preprocessed.shape[1]
    model = Net(input_dim)

    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)


    for epoch in range(5):  # loop over the dataset multiple times
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data
            optimizer.zero_grad()
            outputs = model(inputs)
            mse_loss = criterion(outputs, labels.view(-1, 1))
            rmse_loss = torch.sqrt(mse_loss)  # take the square root of the MSE to get the RMSE
            rmse_loss.backward()  # use the RMSE for backpropagation
            optimizer.step()

            # print statistics
            running_loss += mse_loss.item()  # this is still the MSE, not the RMSE
        avg_mse = running_loss / (i+1)
        rmse = torch.sqrt(torch.tensor(avg_mse))
        print(f'Epoch: {epoch+1}, RMSE Loss: {rmse}')

    print('Finished Training')
    
    df_test = dfs_test[taxi_id]
    X = df_test.drop(columns=['TRIP_ID','ORIGIN_CALL','ORIGIN_STAND', 'TAXI_ID', 'TIMESTAMP', 'MISSING_DATA'])

    X_train_preprocessed = preprocessor.fit_transform(X_train).toarray()  # convert to dense array here
    X_test_preprocessed = preprocessor.transform(X).toarray()  # convert to dense array here

    test_data = torch.tensor(X_test_preprocessed, dtype=torch.float)
    test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

    model.eval()
    predictions = []

    with torch.no_grad():
        for inputs in test_loader:
            outputs = model(inputs)
            predictions.extend(outputs.numpy())

    predictions = [arr.item() for arr in predictions]
    
    total[taxi_id] = predictions
    



  0%|                                                                                          | 0/320 [00:00<?, ?it/s]

taxi_id: 20000542,index: 0
Epoch: 1, RMSE Loss: 695.0496215820312
Epoch: 2, RMSE Loss: 627.8548583984375
Epoch: 3, RMSE Loss: 626.2178955078125
Epoch: 4, RMSE Loss: 622.5540771484375


  0%|▎                                                                                 | 1/320 [00:04<24:40,  4.64s/it]

Epoch: 5, RMSE Loss: 623.7300415039062
Finished Training
taxi_id: 20000108,index: 1
Epoch: 1, RMSE Loss: 606.336669921875
Epoch: 2, RMSE Loss: 534.1712646484375
Epoch: 3, RMSE Loss: 531.9000854492188
Epoch: 4, RMSE Loss: 532.809326171875


  1%|▌                                                                                 | 2/320 [00:09<23:58,  4.52s/it]

...

Epoch: 5, RMSE Loss: 736.5397338867188
Finished Training
taxi_id: 20000667,index: 318
Epoch: 1, RMSE Loss: 618.67919921875
Epoch: 2, RMSE Loss: 513.3304443359375
Epoch: 3, RMSE Loss: 512.5057373046875
Epoch: 4, RMSE Loss: 510.8667297363281


100%|███████████████████████████████████████████████████████████████████████████████▊| 319/320 [17:58<00:03,  3.28s/it]

Epoch: 5, RMSE Loss: 510.6474609375
Finished Training
taxi_id: 20000255,index: 319
Epoch: 1, RMSE Loss: 500.149658203125
Epoch: 2, RMSE Loss: 390.3009948730469
Epoch: 3, RMSE Loss: 391.35406494140625
Epoch: 4, RMSE Loss: 390.9750671386719


100%|████████████████████████████████████████████████████████████████████████████████| 320/320 [18:02<00:00,  3.38s/it]

Epoch: 5, RMSE Loss: 390.4744567871094
Finished Training


In [ ]:
total2

In [34]:
total2 = copy.deepcopy(total)

In [36]:
test_public = df_test = pd.read_csv("test_public.csv")

In [41]:
taxi_trip_id_dic = test_public.set_index('TRIP_ID')['TAXI_ID'].to_dict()

In [51]:
new_dic = {}
for trip_id in taxi_trip_id_dic.keys():
    l = total2[taxi_trip_id_dic[trip_id]]
    new_dic[trip_id] = l[0]
    total2[taxi_trip_id_dic[trip_id]] = total2[taxi_trip_id_dic[trip_id]][1:]

In [53]:
df_sample = pd.read_csv("sampleSubmission.csv")

df_sample["TRAVEL_TIME"] = df_sample['TRIP_ID'].apply(lambda x: new_dic[x])

df_sample.to_csv("my_pred.csv", index=None)